# Segmentation 3D

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [3]:
import cv2

from openalea.phenomenal.data_access import (
    plant_1_voxel_grid,
    plant_2_voxel_grid,
    plant_1_images_binarize)

from openalea.phenomenal.display import *
from openalea.phenomenal.data_structure import (
    VoxelSkeleton,
    VoxelGrid,
    VoxelSegmentation,
    ImageView)

from openalea.phenomenal.multi_view_reconstruction import *

from openalea.phenomenal.segmentation_3D import (
    voxel_graph_from_voxel_grid,
    skeletonize,
    labelize_maize_skeleton,
    maize_analysis,
    segment_reduction)

from openalea.phenomenal.data_access.plant_1 import (
    plant_1_images_binarize,
    plant_1_calibration_camera_side,
    plant_1_calibration_camera_top)


In [4]:
def routine_select_ref_angle(image_views):

    max_len = 0
    image_view_max = None
    for i, iv in enumerate(image_views):

        x_pos, y_pos, x_len, y_len = cv2.boundingRect(cv2.findNonZero(iv.image))

        if x_len > max_len:
            max_len = x_len
            image_view_max = iv

    image_view_max.ref = True

    return image_views

In [6]:
images = plant_1_images_binarize()

calibration_side = plant_1_calibration_camera_side()
calibration_top = plant_1_calibration_camera_top()

# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle], projection, inclusive=False))

image_views = routine_select_ref_angle(image_views)

projection = calibration_top.get_projection(0)
image_views.append(ImageView(images[-1], projection, inclusive=True))

voxels_size = 8
error_tolerance = 1
vg = reconstruction_3d(image_views,
                       voxels_size=voxels_size,
                       error_tolerance=error_tolerance,
                       verbose=True)

vg.voxels_position = map(tuple, list(vg.voxels_position))
vg.voxels_size = int(vg.voxels_size)

print len(vg.voxels_position)
# 27762

DisplayVoxelGrid(vg).show()

Iteration 1 / 9 : 8
Iteration 2 / 9 : 64
Iteration 3 / 9 : 512
Iteration 4 / 9 : 4096
Iteration 5 / 9 : 400
Iteration 6 / 9 : 1392
Iteration 7 / 9 : 4216
Iteration 8 / 9 : 13776
Iteration 9 / 9 : 55560
27762


In [ ]:
import time
t0 = time.time()

subgraph = None
voxel_graph = voxel_graph_from_voxel_grid(vg, connect_all_point=True)
voxel_skeleton = skeletonize(voxel_graph.graph,
                             voxel_graph.voxels_size,
                             subgraph=subgraph)

In [ ]:
# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle],
                                 projection,
                                 inclusive=False))

print time.time() - t0, "s"
t0 = time.time()
print len(voxel_skeleton.voxel_segments)
DisplaySkeleton(voxel_skeleton).show()
voxel_skeleton = segment_reduction(voxel_skeleton,
                                   image_views,
                                   tolerance=4)

In [ ]:
print time.time() - t0, "s"
print len(voxel_skeleton.voxel_segments)
DisplaySkeleton(voxel_skeleton).show()

vms = labelize_maize_skeleton(voxel_skeleton, voxel_graph)
filename = 'voxel_maize_segmentation.json'
vms.write_to_json_gz(filename)
vms = VoxelSegmentation.read_from_json_gz(filename)
DisplaySegmentation(vms).show()

In [7]:
vmsi = maize_analysis(vms)
file_prefix = 'voxel_maize_segmentation_info'
vmsi.write_to_json_gz(file_prefix)
vmsi = VoxelSegmentation.read_from_json_gz(file_prefix)

DisplaySegmentation(vmsi).show(1)

23.4359998703 s
34
489.663000107 s
15
